In [1]:
# 모듈 로딩
import torch
from torchinfo import summary
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

# 경로 지정 모듈 로딩
import sys
sys.path.append(r'C:\Users\KDP-50\OneDrive\바탕 화면\KDT_DYH\14.personal_project(final)2')

import os
from NLP_Class import LSTMClassifier

In [2]:
def check_path(path):
    if os.path.exists(path):
        print(f'{path} 경로가 존재.')
    else:
        print(f'{path} 경로가 존재 X, 재설정 요')

In [3]:
# 저장 경로
SAVE_PATH='./LSTM/'
SAVE_MODEL=SAVE_PATH+'LSTM_9_loss(0.05198).pth'

check_path(SAVE_PATH)
check_path(SAVE_MODEL)

./LSTM/ 경로가 존재.
./LSTM/LSTM_9_loss(0.05198).pth 경로가 존재.


In [4]:
NLP_model=torch.load(SAVE_MODEL, weights_only=False)

In [5]:
print(NLP_model)

LSTMClassifier(
  (lstm): LSTM(5000, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=2, bias=True)
)


In [6]:
summary(NLP_model)

Layer (type:depth-idx)                   Param #
LSTMClassifier                           --
├─LSTM: 1-1                              2,626,560
├─Linear: 1-2                            258
Total params: 2,626,818
Trainable params: 2,626,818
Non-trainable params: 0

- 모델 테스트

In [7]:
# 1. 테스트용 데이터

sample_texts = [
    "안녕하세요 배송이 되지 않아서요 배송이 되는걸까요? 배송이 된다면 택배사와 송장번호 알려주세요.",
    "일요일에 주문한걸 수요일까지 발송도 안해놓구 배송지연으로 취소하니 목요일에서야 답변으로 오늘 출고예정이라며 이미 배송시작이라고 취소 불가라니 조회해보니 발송예정인데 어이없네요",
    "레고 정품 아니고 중국제 짭입니다 박스사진만 저렇게 해놓고 내용물은 짭입니다 이건 사기 입니다.",
    "상품 가격은 올려놓고 출고가 늦어질 것 같아서 취소한다니 기분이 영 그렇네요"
]

In [8]:
# 2. 데이터 전처리 (TF-IDF 벡터화)

# 저장된 Vectorizer 로드
tfidf_vectorizer = joblib.load('./LSTM/tfidf_vectorizer.pkl')

# 샘플 데이터 TF-IDF 벡터 변환
sample_vectors = tfidf_vectorizer.transform(sample_texts).toarray()

In [9]:
# 3. 학습된 모델 로딩
NLP_model.eval()

# 4. 모델 예측
sample_tensor = torch.tensor(sample_vectors, dtype=torch.float32)
predictions = NLP_model(sample_tensor).squeeze()

In [10]:
predicted_labels = predictions.argmax(dim=1)

In [12]:
predicted_labels

tensor([1, 0, 1, 0])

In [11]:
# 5. 결과 출력
for text, pred in zip(sample_texts, predicted_labels):
    result = "어뷰징" if pred.item() == 0 else "정상"
    print(f"리뷰: {text}\n예측: {result}\n")

리뷰: 안녕하세요 배송이 되지 않아서요 배송이 되는걸까요? 배송이 된다면 택배사와 송장번호 알려주세요.
예측: 정상

리뷰: 일요일에 주문한걸 수요일까지 발송도 안해놓구 배송지연으로 취소하니 목요일에서야 답변으로 오늘 출고예정이라며 이미 배송시작이라고 취소 불가라니 조회해보니 발송예정인데 어이없네요
예측: 어뷰징

리뷰: 레고 정품 아니고 중국제 짭입니다 박스사진만 저렇게 해놓고 내용물은 짭입니다 이건 사기 입니다.
예측: 정상

리뷰: 상품 가격은 올려놓고 출고가 늦어질 것 같아서 취소한다니 기분이 영 그렇네요
예측: 어뷰징



In [ ]:
text_list = []
result_list = []
for text, pred in zip(sample_texts, predicted_labels):
    result = "어뷰징" if pred.item() == 0 else "정상"
    result_list.append(result)
    text_list.append(text)
text_list, result_list

(['안녕하세요 배송이 되지 않아서요 배송이 되는걸까요? 배송이 된다면 택배사와 송장번호 알려주세요.',
  '일요일에 주문한걸 수요일까지 발송도 안해놓구 배송지연으로 취소하니 목요일에서야 답변으로 오늘 출고예정이라며 이미 배송시작이라고 취소 불가라니 조회해보니 발송예정인데 어이없네요',
  '레고 정품 아니고 중국제 짭입니다 박스사진만 저렇게 해놓고 내용물은 짭입니다 이건 사기 입니다.',
  '상품 가격은 올려놓고 출고가 늦어질 것 같아서 취소한다니 기분이 영 그렇네요'],
 ['정상', '어뷰징', '정상', '어뷰징'])

In [17]:
for text in text_list:
    print('리뷰:',text)

리뷰: 안녕하세요 배송이 되지 않아서요 배송이 되는걸까요? 배송이 된다면 택배사와 송장번호 알려주세요.
리뷰: 일요일에 주문한걸 수요일까지 발송도 안해놓구 배송지연으로 취소하니 목요일에서야 답변으로 오늘 출고예정이라며 이미 배송시작이라고 취소 불가라니 조회해보니 발송예정인데 어이없네요
리뷰: 레고 정품 아니고 중국제 짭입니다 박스사진만 저렇게 해놓고 내용물은 짭입니다 이건 사기 입니다.
리뷰: 상품 가격은 올려놓고 출고가 늦어질 것 같아서 취소한다니 기분이 영 그렇네요


In [15]:
# 1. 테스트용 데이터

sample_texts2 = [
    "9월 27일 주문건 아직도 지연중~~ 취소함배송이 오래 걸리면 상세페이지에 솔직하게 적어놓으세요~~7 ~ 14일 정도 걸린다고 하지 마시고한달이상 걸린다고 솔직하게 적어놓으세요장난하는것도 아니고 사람 피말리게 만드네",
    "저희 법무팀으로 연락을 주시기 바랍니다",
    "지연연락도 없고. 취소도 맘데로 하셧네요??첨 주문 햇던건데~ 두번은 없을듯요.",
    "물건이 떡이되어오고.. 스트로폼은 손상된걸 테이프붙여서 보내고... 상한거먹으라는건가요??반품요청했습니다"
]

# 2. 데이터 전처리 (TF-IDF 벡터화)

# 저장된 Vectorizer 로드
tfidf_vectorizer = joblib.load('./LSTM/tfidf_vectorizer.pkl')

# 샘플 데이터 TF-IDF 벡터 변환
sample_vectors = tfidf_vectorizer.transform(sample_texts2).toarray()

# 3. 학습된 모델 로딩
NLP_model.eval()

# 4. 모델 예측
sample_tensor = torch.tensor(sample_vectors, dtype=torch.float32)
predictions = NLP_model(sample_tensor).squeeze()

predicted_labels = predictions.argmax(dim=1)

# 5. 결과 출력
for text, pred in zip(sample_texts2, predicted_labels):
    result = "어뷰징" if pred.item() == 0 else "정상"
    print(f"리뷰: {text}\n예측: {result}\n")

리뷰: 9월 27일 주문건 아직도 지연중~~ 취소함배송이 오래 걸리면 상세페이지에 솔직하게 적어놓으세요~~7 ~ 14일 정도 걸린다고 하지 마시고한달이상 걸린다고 솔직하게 적어놓으세요장난하는것도 아니고 사람 피말리게 만드네
예측: 어뷰징

리뷰: 저희 법무팀으로 연락을 주시기 바랍니다
예측: 정상

리뷰: 지연연락도 없고. 취소도 맘데로 하셧네요??첨 주문 햇던건데~ 두번은 없을듯요.
예측: 어뷰징

리뷰: 물건이 떡이되어오고.. 스트로폼은 손상된걸 테이프붙여서 보내고... 상한거먹으라는건가요??반품요청했습니다
예측: 정상

